# 嘴唇上色

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# 初始化MediaPipe面部偵測器和面部標誌模型
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

lip_u=[0,37,39,185,80,81,82,13,312,311,310,409,270,269,267]

lip_l=[14,87,178,88,95,146,91,181,84,17,314,405,321,375,324,318,402,317]

# 定義化妝效果函數
def apply_lipstick(image, landmarks, color):
    
    lips_points_u = np.array([landmarks[i] for i in lip_u])  # 嘴唇的點
    lips_points_l = np.array([landmarks[i] for i in lip_l])  # 嘴唇的點
    
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, [lips_points_u], color)
    cv2.fillPoly(mask, [lips_points_l], color)
    result = cv2.addWeighted(image, 1, mask, 0.4, 0)
    return result
imagepath = './256img_lst/'
filename = '006.jpg'
# 讀取圖像
cap = cv2.imread(imagepath +'/' +filename)
# 開啟攝像頭
# cap = cv2.VideoCapture(0)

# 獲取攝像頭的幀寬和幀高
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# creat track bars
def nothing(x):
    pass
cv2.namedWindow('Bar')
cv2.createTrackbar('bar_R', 'Bar', 0, 255, nothing)   # in 'Track Bar' windows
cv2.createTrackbar('bar_G', 'Bar', 0, 255, nothing)   # in 'Track Bar' windows
cv2.createTrackbar('bar_B', 'Bar', 0, 255, nothing)   # in 'Track Bar' windows

while cap.isOpened():
    bar_red = cv2.getTrackbarPos('bar_R', 'Bar')
    bar_green = cv2.getTrackbarPos('bar_G', 'Bar')
    bar_blue = cv2.getTrackbarPos('bar_B', 'Bar')
    
    ret, frame = cap.read()
    if not ret:
        break
    frame=cv2.flip(frame, 1)
    # 偵測面部
    results = face_mesh.process(frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 提取面部特徵點
            height, width, _ = frame.shape
            landmarks = [(int(point.x * width), int(point.y * height)) for point in face_landmarks.landmark]

            # 應用化妝效果（例如口紅）
            frame = apply_lipstick(frame, landmarks, (bar_blue, bar_green, bar_red))  # 紅色口紅

    # 顯示結果影像
    cv2.imshow('Bar', frame)
    
    if cv2.waitKey(10) & 0xFF == 27:
        print("Detected 'esc' key press. Exiting loop.")
        break


cv2.destroyAllWindows()

# 釋放資源
face_mesh.close()

AttributeError: 'numpy.ndarray' object has no attribute 'get'

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh and drawing utils
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

lip_u = [0, 37, 39, 185, 80, 81, 82, 13, 312, 311, 310, 409, 270, 269, 267]
lip_l = [14, 87, 178, 88, 95, 146, 91, 181, 84, 17, 314, 405, 321, 375, 324, 318, 402, 317]

# Define the makeup effect function
def apply_lipstick(image, landmarks, color,const = 0.9 ):
    lips_points_u = np.array([landmarks[i] for i in lip_u])  # Upper lip points
    lips_points_l = np.array([landmarks[i] for i in lip_l])  # Lower lip points

    # Get color data for upper lip points
    upper_lip_colors = [image[y, x] for (x, y) in lips_points_u]

    # Get color data for lower lip points
    lower_lip_colors = [image[y, x] for (x, y) in lips_points_l]
    
    mask = image.copy()
    cv2.fillPoly(mask, [lips_points_u], color)
    cv2.fillPoly(mask, [lips_points_l], color)
    # Convert the image to grayscale
    gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

# Parameters for the GuidedFilter
    radius = 10         # Radius of the guided filter
    eps = 0.1         # Regularization parameter (epsilon)

    guided_filter = cv2.ximgproc.createGuidedFilter(guide=gray_mask, radius=radius, eps=eps)
    filtered_mask = guided_filter.filter(mask)

    result = cv2.addWeighted(image, const, filtered_mask, 1-const, 0)

    return result

# Create trackbars to adjust lip color
def nothing(x):
    pass

cv2.namedWindow('Trackbars')
cv2.createTrackbar('R', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('G', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('B', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('level', 'Trackbars', 0, 100, nothing)

# Path to the image
image_path = './256img_lst/006.jpg'

# Read the image
image = cv2.imread(image_path)
if image is None:
    print(f"Failed to load image: {image_path}")
    exit()

# Detect face in the image
results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        height, width, _ = image.shape
        landmarks = [(int(point.x * width), int(point.y * height)) for point in face_landmarks.landmark]

while True:
    # Get the current positions of the trackbars
    r = cv2.getTrackbarPos('R', 'Trackbars')
    g = cv2.getTrackbarPos('G', 'Trackbars')
    b = cv2.getTrackbarPos('B', 'Trackbars')
    const = cv2.getTrackbarPos('level', 'Trackbars')
    const = -(float(const)-50)/500+0.9
    img = np.zeros((100, 100, 3), np.uint8) 
    img[:] = [b,g,r]
    # Apply the makeup effect with the current color
    if results.multi_face_landmarks:
        output_image = apply_lipstick(image.copy(), landmarks, (b, g, r),const)
    else:
        output_image = image.copy()

    # Display the result
    cv2.imshow('Lipstick Application', output_image)
    cv2.imshow('color plate',img)
    # Exit on 'esc' key press
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
face_mesh.close()

c:\Users\User\anaconda3\envs\py_39_cv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


# 眼妝

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh and drawing utils
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


# Define the makeup effect function
def apply_eyeshade(image, mask, color,const = 0.9 ):

    mask[mask[:]!=(0,0,0)]    
    cv2.fillPoly(mask,  color)
    cv2.fillPoly(mask,  color)
    # Convert the image to grayscale
    gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

# Parameters for the GuidedFilter
    radius = 10         # Radius of the guided filter
    eps = 0.1         # Regularization parameter (epsilon)

    guided_filter = cv2.ximgproc.createGuidedFilter(guide=gray_mask, radius=radius, eps=eps)
    filtered_mask = guided_filter.filter(mask)

    result = cv2.addWeighted(image, const, filtered_mask, 1-const, 0)

    return result

# Create trackbars to adjust lip color
def nothing(x):
    pass

cv2.namedWindow('Trackbars')
cv2.createTrackbar('R', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('G', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('B', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('level', 'Trackbars', 0, 100, nothing)

# Path to the image
image_path = './256img_lst/006.jpg'
rwmask_path ='./pymatting_outcome_rw/006.jpg'

# Read the image
image = cv2.imread(image_path)
if image is None:
    print(f"Failed to load image: {image_path}")
    exit()

# Detect face in the image
results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        height, width, _ = image.shape
        landmarks = [(int(point.x * width), int(point.y * height)) for point in face_landmarks.landmark]

while True:
    # Get the current positions of the trackbars
    r = cv2.getTrackbarPos('R', 'Trackbars')
    g = cv2.getTrackbarPos('G', 'Trackbars')
    b = cv2.getTrackbarPos('B', 'Trackbars')
    const = cv2.getTrackbarPos('level', 'Trackbars')
    const = -(float(const)-50)/500+0.9
    img = np.zeros((100, 100, 3), np.uint8) 
    img[:] = [b,g,r]
    # Apply the makeup effect with the current color
    if results.multi_face_landmarks:
        output_image = apply_lipstick(image.copy(), landmarks, (b, g, r),const)
    else:
        output_image = image.copy()
    
    # Display the result
    cv2.imshow('Lipstick Application', output_image)
    cv2.imshow('color plate',img)

    # Exit on 'esc' key press
    if cv2.waitKey(1) & 0xFF == 27:
        break
cv2.destroyAllWindows()
face_mesh.close()

In [8]:
import cv2
import mediapipe as mp
import numpy as np
rwmask_path ='./pymatting_outcome_rw/006.jpg'
mask = cv2.imread(rwmask_path)

mask1 = np.all(mask != (0, 0, 0), axis=-1)
mask[mask1] = (127,127,255)
cv2.imshow('Modified Mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()
